# Data cleaning 

In [109]:
import pandas as pd

In [110]:
from use_cases.contributions import create_table_contributions

from use_cases.emotions import create_table_emotions
from use_cases.personal_needs import create_table_personal_needs
from use_cases.dialogues import create_table_dialogues
from use_cases.country_needs import create_table_country_needs
from use_cases.persons import create_table_persons
from use_cases.individuals import create_table_individuals


pd.options.mode.chained_assignment = None 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data Surveys

In [111]:
chunksize   = 100 # Just for debugging
survey_path = './data/BBDD_Dialogos_2021_01_27.csv' 
survey = pd.read_csv(survey_path, chunksize=chunksize, low_memory=False)
for survey in survey: break
    
ind_online_survey_path = './data/Consulta_Individual_online_v2.xlsx'
ind_online_survey = pd.read_excel(ind_online_survey_path, 'Sheet1')
    
ind_survey_path = './data/Base_final_digitación_Consultas.xlsx'
ind_survey = pd.read_excel(ind_survey_path, 'CONSULTAS')

In [112]:
survey.sample(2)
ind_online_survey.sample(2)

,Submission Date,RUN,Edad,Comuna,1. ¿Cuál es el nivel de educación alcanzado por Usted?,2. ¿Cuál es el nivel de educación que alcanzó el jefe de su hogar? (sólo si se trata de alguien distinto a Usted),1 >> Emociones / Sentimientos / Sensaciones,1 >> Explique lo mencionado,2 >> Emociones / Sentimientos / Sensaciones,2 >> Explique lo mencionado,...,2 >> Rol del actor social (Describa),3 >> Necesidades del país identificadas,3 >> Rol del Estado (Describa),"3 >> Actor social (empresa, organizaciones sociales, medios de comunicación, comunidad, etc)",3 >> Rol del actor social (Describa),5. Pregunta: ¿Cuál es mi contribución personal para construir el Chile que queremos?,Comuna.1,"Comentarios finales: Si tiene alguna observación o sugerencia, en este espacio puede indicarla.",Región,Región.1
604,2020/01/10 19:20:18,fc7324c9d73156c4648fa916cec247f4,30,Isla de Pascua,Universitaria completa.,Universitaria completa.,"Bien, sin problemas. Soy muy feliz",Todo en perfecto estado,NaN,NaN,...,Brindar información relevante sobre la materia...,Mejorar el sistema de pensiones,Preponderante,AFP y demás instituciones relacionadas,Involucrados directos en el resultado final de...,Todos los días pretendo desarrollar una labor ...,NaN,"Espero que Chile mejore. Con todo, me encuentr...",Región de Valparaíso,NaN
830,2019/12/30 14:54:04,d66768ef775f80022c4218bbbb508996,39,Macul,Universitaria completa.,Universitaria completa.,Insegura,No hay respuesta concreta a las demandas del p...,Vulnerada,He visto cuanta desigualdad debemos vivir los ...,...,"Este rol debe ser asegurado por el Estado, no ...",Corrupcion,El estado debe castigar la corrupcion desde el...,Tolerancia cero con las practicas corruptas,Castigar moralmente la corrupcion,"Seguir en la lucha, tratar con dignidad y resp...",NaN,NaN,Región Metropolitana de Santiago,NaN


## Processing Uses Cases
1. Emotions 
2. Contributions
3. Country needs 
4. Personal/familiar needs

### Persons Table (Dialogue)

In [69]:
persons_table = create_table_persons(survey)

In [70]:
persons_table.sample(3)

,diag_id,age,sex,level,comuna_id,age_range
36,enc_u_4598385550153533853,21,f,basica_completa,7101,15-30
90,enc_u_4595944093321943387,28,f,universitaria_completa,14101,15-30
6,enc_u_4602649914809752187,21,f,universitaria_incompleta,14101,15-30


### Dialogues

In [67]:
dialogues_table = create_table_dialogues(survey)

In [68]:
dialogues_table.sample(3)

,diag_id,date,init_time,end_time,location,address,comuna_id,n_members,group_name
51,enc_u_4598337190995449677,22-01-2020,17:00:00,00:00:00,direccion regional injuv coquimbo,av francisco de aguirre #414,4101,7,usuarios de oficina movil de bienes nacionales
86,enc_u_4596538444129461689,06-03-2020,16:00:00,17:30:00,terminal lincosur,sala de star terminales,4101,8,usuarios de oficina movil de bienes nacionales
62,enc_u_4597641730116574573,05-03-2020,10:00:00,12:00:00,centro cultural de freire,pedro canales numero 137,9105,5,usuarios de oficina movil de bienes nacionales


### Emotions

<img src="diagrams/q1.png" width=730 height=520 />

In [63]:
%%time
emotions_table = create_table_emotions(survey)

CPU times: user 722 ms, sys: 52 ms, total: 774 ms
Wall time: 774 ms


In [64]:
emotions_table.sample(2)

,diag_id,name,name_token,macro,text,text_tokens
86,ENC_U_4596538444129461689,angustia y preocupacion,"[angustia, preocupacion]",angustia y preocupacion,por la baja en los salarios de las recaudacion...,"[baja, salarios, recaudaciones, diarias, depen..."
73,ENC_U_4597390948805369662,identificada,[identificada],identificada,- participante activa\n- positivo para todas\n...,"[participante, activa, positivo, todas, decepc..."


### Individuals

In [113]:
%%time
individuals_table = create_table_individuals(ind_online_survey, ind_survey)

CPU times: user 8.45 s, sys: 64.3 ms, total: 8.52 s
Wall time: 8.45 s


In [116]:
individuals_table.sample(2)

,id,date,age,comuna_id,level,online,age_range
4375,5078,28-01-2020,19,4201,Media completa. Técnica incompleta,,15-30
12932,14470,03-02-2020,22,3101,Media incompleta (incluyendo Media Técnica),,15-30


### Country Needs

In [118]:
country_needs = create_table_country_needs(survey, ind_survey_path, ind_online_survey_path)

                                   RUN 1 >> Necesidad que enfrenta el país  \
0     258f69b8147009aa54f785bb370d16a2                               salud   
1     aedde5c4c38d225c040a6bc498683b02                           Educación   
2     fab86d9f878e864fc80dc5c3ae82a502                educación de calidad   
3     fc8130c7e264e76fd2330384f787820a          Descentralización de poder   
4     8d2d0d080647bfa6faced5c3c0dd56c2                  Nueva Constitución   
...                                ...                                 ...   
1799  0509e2ef02a033fc0efab1214e68be09                           Seguridad   
1800  c4960481a3875eb1cfbe32c33dc362c0                                 Paz   
1801  2eac9b65b2b1ec5d134d6b45c83315fe                           seguridad   
1802  ddc84f9d6d278cf3d32a0df820e01688          reestablecimiento de orden   
1803  b1e9ef4111f4267cf1df2ca921aadfc8                       Orden Publico   

                          1 >> Explique lo mencionado.1  \
0   

In [62]:
country_needs.sample(3)

,id,diag_id,name,name_tokens,macro,exp,exp_tokens,role,role_tokens,actor,priority
69,520,enc_u_4597425290772173197,amor,[amor],,"- falta de amor por nuestro pais, amor propia,...","[falta, amor, pais, amor, propia, amor, progimo]",- amor por las personas\n- ensenar a amar nues...,"[amor, personas, ensenar, amar, pais]",estado,0
122,424,enc_u_4598360210156649032,medicamentos,[medicamentos],,,,-tener mas fiscalizacion para que los remedio...,"[tener, mas, fiscalizacion, remedios, precios,...",estado,
85,536,enc_u_4596619275164164128,educacion,[educacion],,- fortalecer la educacion publica. \n- no pued...,"[fortalecer, educacion, publica, pueden, forta...",tiene que hacerse cargo de la educacion y debe...,"[hacerse, cargo, educacion, debe, ser, publica]",estado,0


### Family/PersonalNeeds

In [53]:
personal_needs = create_table_personal_needs(survey)

In [54]:
personal_needs.sample(3)

,id,diag_id,name,name_tokens,exp,exp_tokens,macro,priority
67,167,ENC_U_4597430940772627274,salud,[salud],que la salud no sea burocratica y que su atenc...,"[salud, burocratica, atencion, semejante, salu...",salud,2
60,460,ENC_U_4597643359322725615,respuesta sin completar.,"[respuesta, completar]",,,respuesta sin completar.,0
1,1,ENC_U_4602778880117363308,agua potable,"[agua, potable]",aun no tenemos agua potable en el sector rural,"[aun, agua, potable, sector, rural]",agua potable,1


### Contributions

In [58]:
contributions = create_table_contributions(survey)

In [59]:
contributions.sample(3)

,id,diag_id,text,text_tokens,macro
15,415,ENC_U_4601870260112596575,ser buenos ciudadanos.,"[ser, buenos, ciudadanos]",ser buenos ciudadanos.
61,261,ENC_U_4597643308415751967,participar de los procesos electorales que afr...,"[participar, procesos, electorales, afrontara,...",participar de los procesos electorales que afr...
35,135,ENC_U_4598389170159296566,participando en actividades actividades civicas,"[participando, actividades, actividades, civicas]",participando en actividades actividades civicas
